# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [1]:
import os
import sys
import urllib.request
import requests

news_data = []

page_count = 3


client_id = "************"
client_secret = "*********"
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
    
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + '&start=' + str(idx*10+1) # json 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과


    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()


    if(rescode==200):
    #     response_body = response.read()
        result = requests.get(response.geturl(),
                               headers = {'X-Naver-Client-Id' : client_id,
                                          'X-Naver-client-Secret' : client_secret})

        news_data.append(result.json())

    #     print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

### 네이버 OPEN API를 통해 가져온 데이터 확인하기.

In [2]:
len(news_data)

3

In [3]:
# print(news_data[0]['items'][0]['link'])
print(len(news_data))

3


### 가져온 URL이 네이버 뉴스인지 확인하기.

In [4]:
naver_news_link = []

for page in news_data:
    

    page_news_link = []

    for item in page['items']:
        link = item['link']
        if 'naver' in link:
            page_news_link.append(link)

    naver_news_link.append(page_news_link)

for page in naver_news_link:
    for link in page:
        print(link)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004838329
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002212286
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002212265
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=008&aid=0004534787
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002925514
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=016&aid=0001785357
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002211860
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=417&aid=0000649498


In [5]:
page_news_link

['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002211860',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=417&aid=0000649498']

In [6]:
naver_news_link

[['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004838329',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002212286',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002212265'],
 ['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=008&aid=0004534787',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002925514',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=016&aid=0001785357'],
 ['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002211860',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=417&aid=0000649498']]

### 네이버 뉴스 기사 크롤링!

In [7]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm.notebook import tqdm
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import time

In [8]:
os.getcwd()

'D:\\데이터분석\\국민대\\2021 겨울 특강_패스트캠퍼스\\3.머신러닝과 데이터분석 올인원\\Part 13~17) Project\\04. 텍스트마이닝을 활용한 카카오톡 감성분석 Project\\실습코드'

In [9]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome('D://데이터분석//chromedriver.exe')

In [12]:
driver.get('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=417&aid=0000649498')

WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.104)


In [44]:
print(naver_news_link[n][idx])

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=417&aid=0000649498


In [11]:
naver_news_title = []
naver_news_content = []


for n in tqdm(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm(range(len(naver_news_link[n]))):
        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])
        

Timeout!
Timeout!
Timeout!



Timeout!
Timeout!
Timeout!



Timeout!
Timeout!


[]
[]


### 크롤링 결과 확인하기.

In [38]:
print(naver_news_title[0])

[]


In [ ]:
print(naver_news_content[0])

In [ ]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

### 크롤링 결과 저장하기.

In [ ]:
with open('naver_news_title.pk','wb') as f:
    pickle.dump(naver_news_title, f)

with open('naver_news_content.pk', 'wb') as f:
    pickle.dump(naver_news_content, f)